Uploaded Kaggle.json

In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
%cd /content
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download lavaman151/plantifydr-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
/content
100% 2.58G/2.58G [00:33<00:00, 35.3MB/s]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,GlobalAveragePooling2D, Conv2D,InputLayer
from tensorflow.keras.layers import BatchNormalization,Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import InceptionResNetV2
import keras.backend as K
from tensorflow.keras.applications import MobileNetV2

from tensorflow import keras

from tensorflow.keras.models import load_model

In [ ]:
TRAINING_DATA_PATH = "/content/drive/MyDrive/HAICK/challenge3"

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 64
LEARNING_RATE = 0.009
MOMENTUM = 0.9 
EPOCHS = 20
PATIENCE = 30
#CLASSES = len(glob('data/train'))

The following model is basically a MobileNet model that has been pretrained on PlantVillage Data (https://www.kaggle.com/thunder2901/leaf-disease-classification-mobilenet/data )

In [ ]:
base_model = keras.applications.MobileNet(weights="imagenet",include_top=False,input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))


17235968/17225924 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv1 (Conv2D)              multiple                  864       
                                                                 
 conv1_bn (BatchNormalizatio  multiple                 128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 128, 128, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 128, 128, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 128, 128, 32)     128       
 ation)                                         

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train = train_datagen.flow_from_directory(
    TRAINING_DATA_PATH,
    target_size=(256,256),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training') # set as training data

valid = train_datagen.flow_from_directory(
    TRAINING_DATA_PATH, # same directory as training data
    target_size=(256,256),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation') # set as validation data



Found 1395 images belonging to 3 classes.
Found 348 images belonging to 3 classes.


In [ ]:
inputShape = (256, 256, 3)
model=Sequential()

model.add(InputLayer(input_shape=inputShape))
  
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(0.5,name='Dropout_1'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu',name='fc2'))
model.add(Dropout(0.3,name = 'Dropout'))
model.add(Dense(3,activation = 'softmax',name='output'))

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (Dense)                 (None, 256, 256, 1024)    4096      
                                                                 
 Dropout_1 (Dropout)         (None, 256, 256, 1024)    0         
                                                                 
 batch_normalization_7 (Batc  (None, 256, 256, 1024)   4096      
 hNormalization)                                                 
                                                                 
 fc2 (Dense)                 (None, 256, 256, 512)     524800    
                                                                 
 Dropout (Dropout)           (None, 256, 256, 512)     0         
                                                                 
 output (Dense)              (None, 256, 256, 1)       513       
                                                      

In [ ]:
adam=optimizers.Adam(learning_rate=0.001)
model.compile(
        loss = 'categorical_crossentropy',
        optimizer = adam,
        metrics = ['accuracy']
  )

In [ ]:
train

In [ ]:
model.fit_generator(
    train,
    steps_per_epoch = train.samples // BATCH_SIZE,
    validation_data = valid, 
    validation_steps = valid.samples // BATCH_SIZE,
    epochs = 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10


ValueError: ignored

In [ ]:
y_pred=model.predict(valid)
y_pred=np.argmax(y_pred,axis=1)
y_true=valid.classes


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true,y_pred)

0.11356013416387159